# Implementando LGBM como regresor
- Alvarado Morán Óscar
- Bermúdez Marbán Dante

In [1]:
import numpy as np

import pandas as pd

#!conda install -c conda-forge lightgbm -y
import lightgbm as lgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

## Usando algo así como random search

MAE:  2580211.5837444784

{'learning_rate': 0.24842943048773125, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.6330313130909996, 'num_leaves': 48, 'min_data': 21, 'max_depth': 27}

---

MAE: 2553720.985319375

{'learning_rate': 0.1275200906747731,
 'boosting_type': 'dart',
 'objective': 'regression',
 'metric': 'mae',
 'sub_feature': 0.4067123592984939,
 'num_leaves': 287,
 'min_data': 19,
 'max_depth': 94}
 
---
 
MAE: 2547251.821188033

Parámetros: {'learning_rate': 0.14124527912898743, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.5486980094996131, 'num_leaves': 43, 'min_data': 23, 'max_depth': 68}

Iteraciones: 8972

---

MAE: 2671093.1977523025

Parámetros: {'learning_rate': 0.4151061569602352, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.5989341446930334, 'num_leaves': 70, 'min_data': 10, 'max_depth': 97, 'verbose': -1}

In [7]:
def random_search(X_train, y_train, X_test, y_test, its = 50, iters = 10000):
    #Set the minimum error arbitrarily large
    minimo = float("inf") 
    pp = {}
    count = 0 #Used for keeping track of the iteration number
    maes = []
    pars = []
    #How many runs to perform using randomly selected hyperparameters
    iterations = its
    iteraciones = 0
    while count < its:
        print('iteration number', count)
        params = {} #initialize parameters
        try:
            d_train = lgb.Dataset(X_train, label = y_train) #Load in data
            params['learning_rate'] = np.random.uniform(0.4, 0.8)
            params['boosting_type'] = 'dart' # Quitamos 'goss', da malos resultados. Es mejor 'dart' que 'gbdt'.
            params['objective'] = 'regression'
            params['metric'] = 'mae'
            params['sub_feature'] = np.random.uniform(0, 1)
            params['num_leaves'] = np.random.randint(20, 300)
            params['min_data'] = np.random.randint(1, 50) # 100
            params['max_depth'] = np.random.randint(5, 300) # 200
            params['verbose'] = -1
            print(params) #Train using selected parameters
            clf = lgb.train(params, d_train, iters)
            y_pred = clf.predict(X_test) #Create predictions on test set
            mae = mean_absolute_error(y_pred,y_test)    
            print('MAE:', mae)
            print("----"*5)
            maes.append(mae)
            pars.append(params)
            if mae < minimo:
                minimo = mae
                pp = params
            count += 1 #increment count
        except: #in case something goes wrong
            print('failed with---------------------')
            print(params)
    return maes, pars, minimo, pp

In [8]:
%%time
# Con las estadísticas de señales
datos = pd.read_csv("../csvs/stats_per_file_signal_train2.csv")
train = pd.read_csv("/home/oscar/Escritorio/predict-volcanic-eruptions-ingv-oe/train.csv")
datos_time = datos.join(train.set_index("segment_id"), on = "segment_id")

X = datos_time.iloc[:,1:-1].values
y = datos_time.iloc[:,-1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)
maes1, pars1, minimo1, pp1 = random_search(X_train, y_train, X_test, y_test, its = 10)
print(f"MAE: {minimo1}")
print(f"Parámetros: {pp1}")

iteration number 0
{'learning_rate': 0.4391034496782301, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.42830399997570645, 'num_leaves': 166, 'min_data': 39, 'max_depth': 34, 'verbose': -1}
MAE: 2759979.947004503
--------------------
iteration number 1
{'learning_rate': 0.43572346536894696, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.592262706528372, 'num_leaves': 266, 'min_data': 28, 'max_depth': 180, 'verbose': -1}
MAE: 2710302.594225801
--------------------
iteration number 2
{'learning_rate': 0.4845715465489533, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.9874855152563691, 'num_leaves': 131, 'min_data': 39, 'max_depth': 288, 'verbose': -1}
MAE: 2835409.843804503
--------------------
iteration number 3
{'learning_rate': 0.4383395014090739, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.2769628734248536, 'num_leaves': 227, '

In [9]:
gbdts = []
darts = []
for idx, param in enumerate(pars1):
    param['mae'] = maes1[idx]
    if param['boosting_type'] == 'gbdt':
        gbdts.append(param)
    else:
        darts.append(param)

In [20]:
dic_gbdts = {par: [dic[par] for dic in gbdts] for par in gbdts[0].keys() if par not in ['boosting_type', 'verbose',
                                                                                        'objective', 'metric']}
dic_darts = {par: [dic[par] for dic in darts] for par in darts[0].keys() if par not in ['boosting_type', 'verbose',
                                                                                        'objective', 'metric']}
dic_darts.keys()

dict_keys(['learning_rate', 'sub_feature', 'num_leaves', 'min_data', 'max_depth', 'mae'])

In [ ]:
import plotly.graph_objects as go
X_gbdt = dic_gbdts['num_leaves']
Y_gbdt = dic_gbdts['min_data']
Z_gbdt = dic_gbdts['max_depth']
col = dic_gbdts['mae']
s = dic_gbdts['learning_rate']

In [54]:
fig = go.Figure(data=go.Scatter(x = X_gbdt, y = col, mode='markers'))
fig.update_layout(
    title = "Número de hojas GBDT",
    xaxis_title = "Número de hojas",
    yaxis_title = "MAE")
fig.show()

In [55]:
fig = go.Figure(data=go.Scatter(x = Y_gbdt, y = col, mode='markers'))
fig.update_layout(
    title = "min_data GBDT",
    xaxis_title = "min_data",
    yaxis_title = "MAE")
fig.show()

In [56]:
fig = go.Figure(data=go.Scatter(x = Z_gbdt, y = col, mode='markers'))
fig.update_layout(
    title = "Máxima profundidad GBDT",
    xaxis_title = "Máxima profundidad",
    yaxis_title = "MAE")
fig.show()

In [57]:
fig = go.Figure(data=go.Scatter(x = s, y = col, mode='markers'))
fig.update_layout(
    title = "Tasa de aprendizaje GBDT",
    xaxis_title = "Tasa de aprendizaje",
    yaxis_title = "MAE")
fig.show()

In [53]:
fig = go.Figure(data = [go.Scatter3d(x = X_gbdt, y = Y_gbdt, z = Z_gbdt,
                                     mode = 'markers', marker = dict(color = col, size = 3, 
                                                                     colorscale = "hot", 
                                                                     colorbar = {"x": 0.9, 
                                                                                 "y": 0.5, 
                                                                                 "title": "MAE"}))])
fig.show()

In [58]:
X_dart = dic_darts['num_leaves']
Y_dart = dic_darts['min_data']
Z_dart = dic_darts['max_depth']
mae_dart = dic_darts['mae']
lr_dart = dic_darts['learning_rate']

fig = go.Figure(data=go.Scatter(x = X_dart, y = mae_dart, mode='markers'))
fig.update_layout(
    title = "Número de hojas DART",
    xaxis_title = "Número de hojas",
    yaxis_title = "MAE")
fig.show()

In [59]:
fig = go.Figure(data = go.Scatter(x = Y_dart, y = mae_dart, mode='markers'))
fig.update_layout(
    title = "min_data DART",
    xaxis_title = "min_data",
    yaxis_title = "MAE")
fig.show()

In [60]:
fig = go.Figure(data=go.Scatter(x = Z_dart, y = mae_dart, mode='markers'))
fig.update_layout(
    title = "Máxima profundidad DART",
    xaxis_title = "Máxima profundidad",
    yaxis_title = "MAE")
fig.show()

In [61]:
fig = go.Figure(data=go.Scatter(x = lr_dart, y = mae_dart, mode='markers'))
fig.update_layout(
    title = "Tasa de aprendizaje DART",
    xaxis_title = "Tasa de aprendizaje",
    yaxis_title = "MAE")
fig.show()

In [72]:
fig = go.Figure(data = [go.Scatter3d(x = X_dart, y = Y_dart, z = Z_dart,
                                     mode = 'markers', marker = dict(color = mae_dart, size = 3, 
                                                                     colorscale = "hot", 
                                                                     colorbar = {"x": 0.9, 
                                                                                 "y": 0.5, 
                                                                                 "title": "MAE"}))])
fig.show()

In [63]:
%%time
maes2, pars2, minimo2, pp2 = random_search(X_train, y_train, X_test, y_test, its = 200)
print(f"MAE: {minimo2}")
print(f"Parámetros: {pp2}")

iteration number 0
{'learning_rate': 0.6309631502723161, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.4256217417079544, 'num_leaves': 108, 'min_data': 4, 'max_depth': 158, 'verbose': -1} 10000
MAE: 2771389.54705639
--------------------
iteration number 1
{'learning_rate': 0.6261437921518064, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.6191405256674336, 'num_leaves': 40, 'min_data': 32, 'max_depth': 140, 'verbose': -1} 10000
MAE: 2828850.6803586474
--------------------
iteration number 2
{'learning_rate': 0.6605095896353974, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.5736588695432903, 'num_leaves': 127, 'min_data': 38, 'max_depth': 114, 'verbose': -1} 10000
MAE: 2800323.0750027476
--------------------
iteration number 3
{'learning_rate': 0.7790515634134325, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.06519942347506835, 'n

MAE: 2769802.919111955
--------------------
iteration number 30
{'learning_rate': 0.7883295138827046, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.9555522293827101, 'num_leaves': 66, 'min_data': 10, 'max_depth': 18, 'verbose': -1} 10000
MAE: 3048901.8587022466
--------------------
iteration number 31
{'learning_rate': 0.572561242850036, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.9651607952870261, 'num_leaves': 248, 'min_data': 46, 'max_depth': 71, 'verbose': -1} 10000
MAE: 2893881.131546728
--------------------
iteration number 32
{'learning_rate': 0.5487978995060951, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.7133985782458023, 'num_leaves': 167, 'min_data': 32, 'max_depth': 208, 'verbose': -1} 10000
MAE: 2763913.820492765
--------------------
iteration number 33
{'learning_rate': 0.5514362844148065, 'boosting_type': 'dart', 'objective': 'regression', 'metric':

MAE: 2820689.140487201
--------------------
iteration number 60
{'learning_rate': 0.6641370394681447, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.9276595840861527, 'num_leaves': 181, 'min_data': 20, 'max_depth': 137, 'verbose': -1} 10000
MAE: 2851483.695504159
--------------------
iteration number 61
{'learning_rate': 0.47026775872327903, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.20978576993018383, 'num_leaves': 297, 'min_data': 38, 'max_depth': 216, 'verbose': -1} 10000
MAE: 2939618.1383451405
--------------------
iteration number 62
{'learning_rate': 0.6295654563188018, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.09407837446313871, 'num_leaves': 227, 'min_data': 31, 'max_depth': 83, 'verbose': -1} 10000
MAE: 3519958.564386223
--------------------
iteration number 63
{'learning_rate': 0.45153919136801157, 'boosting_type': 'dart', 'objective': 'regression', 'm

MAE: 2792312.8179389043
--------------------
iteration number 90
{'learning_rate': 0.47201374814191716, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.25469109804565493, 'num_leaves': 151, 'min_data': 42, 'max_depth': 23, 'verbose': -1} 10000
MAE: 2857211.7891020337
--------------------
iteration number 91
{'learning_rate': 0.4401803637120284, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.16455277187137207, 'num_leaves': 173, 'min_data': 24, 'max_depth': 266, 'verbose': -1} 10000
MAE: 3025570.7501859646
--------------------
iteration number 92
{'learning_rate': 0.7280505619653539, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.2781747202002971, 'num_leaves': 247, 'min_data': 27, 'max_depth': 188, 'verbose': -1} 10000
MAE: 2867193.9663759596
--------------------
iteration number 93
{'learning_rate': 0.6621552740453158, 'boosting_type': 'dart', 'objective': 'regression', 

MAE: 2974238.4382917467
--------------------
iteration number 120
{'learning_rate': 0.5942153451672982, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.6736391818036725, 'num_leaves': 263, 'min_data': 33, 'max_depth': 252, 'verbose': -1} 10000
MAE: 2776623.4797042697
--------------------
iteration number 121
{'learning_rate': 0.5065358882939008, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.5066748552927522, 'num_leaves': 92, 'min_data': 21, 'max_depth': 190, 'verbose': -1} 10000
MAE: 2695271.8518346413
--------------------
iteration number 122
{'learning_rate': 0.47392811455731776, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.8696287653813851, 'num_leaves': 126, 'min_data': 25, 'max_depth': 77, 'verbose': -1} 10000
MAE: 2752448.8216591873
--------------------
iteration number 123
{'learning_rate': 0.57470649567359, 'boosting_type': 'dart', 'objective': 'regression', '

MAE: 2765707.211079689
--------------------
iteration number 150
{'learning_rate': 0.4459766953303849, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.7780620982216506, 'num_leaves': 144, 'min_data': 5, 'max_depth': 85, 'verbose': -1} 10000
MAE: 2773078.1159782545
--------------------
iteration number 151
{'learning_rate': 0.7612191217451544, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.8489684654643875, 'num_leaves': 206, 'min_data': 9, 'max_depth': 211, 'verbose': -1} 10000
MAE: 2847015.433273644
--------------------
iteration number 152
{'learning_rate': 0.46040089770527637, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.6915281043029702, 'num_leaves': 23, 'min_data': 42, 'max_depth': 6, 'verbose': -1} 10000
MAE: 2865000.1697127223
--------------------
iteration number 153
{'learning_rate': 0.48658601399705537, 'boosting_type': 'dart', 'objective': 'regression', 'met

MAE: 2827085.1330220066
--------------------
iteration number 180
{'learning_rate': 0.6376334949720427, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.31998570495405054, 'num_leaves': 54, 'min_data': 25, 'max_depth': 188, 'verbose': -1} 10000
MAE: 2812207.045495328
--------------------
iteration number 181
{'learning_rate': 0.4118623132187498, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.843911649014062, 'num_leaves': 265, 'min_data': 29, 'max_depth': 199, 'verbose': -1} 10000
MAE: 2734991.343973242
--------------------
iteration number 182
{'learning_rate': 0.40654463949520997, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.9188857062741523, 'num_leaves': 261, 'min_data': 44, 'max_depth': 195, 'verbose': -1} 10000
MAE: 2759560.65017926
--------------------
iteration number 183
{'learning_rate': 0.7464821839655171, 'boosting_type': 'dart', 'objective': 'regression', 'm

In [66]:
len(maes2)

200

In [67]:
darts2 = []
for idx, param in enumerate(pars2):
    param['mae'] = maes2[idx]
    darts2.append(param)
dic_darts2 = {par: [dic[par] for dic in darts2] for par in darts2[0].keys() if par not in ['boosting_type', 'verbose',
                                                                                        'objective', 'metric']}

In [68]:
X_dart2 = dic_darts2['num_leaves']
Y_dart2 = dic_darts2['min_data']
Z_dart2 = dic_darts2['max_depth']
mae_dart2 = dic_darts2['mae']
lr_dart2 = dic_darts2['learning_rate']

fig = go.Figure(data = go.Scatter(x = X_dart2, y = mae_dart2, mode='markers'))
fig.update_layout(
    title = "Número de hojas DART",
    xaxis_title = "Número de hojas",
    yaxis_title = "MAE")
fig.show()


In [69]:
fig = go.Figure(data = go.Scatter(x = Y_dart2, y = mae_dart2, mode='markers'))
fig.update_layout(
    title = "min_data DART",
    xaxis_title = "min_data",
    yaxis_title = "MAE")
fig.show()

In [70]:
fig = go.Figure(data=go.Scatter(x = Z_dart2, y = mae_dart2, mode='markers'))
fig.update_layout(
    title = "Máxima profundidad DART",
    xaxis_title = "Máxima profundidad",
    yaxis_title = "MAE")
fig.show()

In [71]:
fig = go.Figure(data=go.Scatter(x = lr_dart2, y = mae_dart2, mode='markers'))
fig.update_layout(
    title = "Tasa de aprendizaje DART",
    xaxis_title = "Tasa de aprendizaje",
    yaxis_title = "MAE")
fig.show()

## Intentando hacer esto de unir vectores de características

In [3]:
# Con las estadísticas de señales
char_señales = pd.read_csv("../csvs/stats_per_file_signal_train2.csv")
char_estadisticas = pd.read_csv("../csvs/stats_per_file3.csv")
char = char_señales.join(char_estadisticas.set_index("segment_id"), on = "segment_id")
train = pd.read_csv("/home/oscar/Escritorio/predict-volcanic-eruptions-ingv-oe/train.csv")
datos_time2 = char.join(train.set_index("segment_id"), on = "segment_id")
datos_time2.fillna(0, inplace = True)
datos_time2.replace(np.inf, 0, inplace = True)
datos_time2.replace(-np.inf, 0, inplace = True)
datos_time2.head()

,segment_id,sensor_1_zcr,sensor_2_zcr,sensor_3_zcr,sensor_4_zcr,sensor_5_zcr,sensor_6_zcr,sensor_7_zcr,sensor_8_zcr,sensor_9_zcr,...,sensor_4_nanmax,sensor_5_nanmax,sensor_6_nanmax,sensor_7_nanmax,sensor_8_nanmax,sensor_9_nanmax,sensor_10_nanmax,sensor_3_nanstd,sensor_9_nanstd,time_to_eruption
0,405613802,1860,7504,1,2743,7464,2289,1090,4336,1,...,4.547,0.000,3.508,5.664,4.190,4.656,5.168,1.000092,1.000023,13060214
1,1343997378,2789,6060,7927,3858,6396,5135,7378,4192,4644,...,10.660,7.203,5.438,5.984,9.240,6.734,6.805,0.999839,1.000197,33135705
2,866792357,5669,6563,2855,3745,4907,13508,2344,9929,3020,...,8.110,17.730,4.490,15.160,14.920,7.570,8.164,1.000040,1.000089,11377813
3,1979931149,3249,1,3523,5089,7417,14501,8011,10864,3985,...,4.580,4.960,4.863,5.027,5.970,4.438,4.375,1.000236,0.999990,18567908
4,1577280457,2561,7583,2031,3764,304,3685,3140,3433,8944,...,5.770,7.390,11.410,4.715,6.703,7.414,5.490,0.999960,1.000242,1162779


In [4]:
from sklearn.decomposition import PCA

X3 = datos_time2.iloc[:, 1:-1].values
y3 = datos_time2.iloc[:, -1].values
pca = PCA(n_components = 10)
principalComponents = pca.fit_transform(X3)
#X_train3, X_test3, y_train3, y_test3 = train_test_split(principalComponents, y3, test_size = 0.25, random_state = 0)
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, test_size = 0.25, random_state = 0)

In [5]:
%%time
maes3, pars3, minimo3, pp3 = random_search(X_train3, y_train3, X_test3, y_test3, its = 10, iters = 5000)
print(f"MAE: {minimo3}")
print(f"Parámetros: {pp3}")

iteration number 0
{'learning_rate': 0.6638528320223085, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.04512156150557034, 'num_leaves': 134, 'min_data': 20, 'max_depth': 161, 'verbose': -1}
MAE: 5399820.111465914
--------------------
iteration number 1
{'learning_rate': 0.6709065725586647, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.5606567952248405, 'num_leaves': 197, 'min_data': 28, 'max_depth': 105, 'verbose': -1}
MAE: 2994083.3477419997
--------------------
iteration number 2
{'learning_rate': 0.5894467990783017, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.17029515262628903, 'num_leaves': 68, 'min_data': 8, 'max_depth': 224, 'verbose': -1}
MAE: 3432424.1641732715
--------------------
iteration number 3
{'learning_rate': 0.5039679549019, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.8482696995898675, 'num_leaves': 233, 'm

In [105]:
pca = PCA(n_components = 10)
principalComponents = pca.fit_transform(X3)
X_train4, X_test4, y_train4, y_test4 = train_test_split(principalComponents, y, test_size = 0.25, random_state = 0)


In [106]:
%%time
maes4, pars4, minimo4, pp4 = random_search(X_train4, y_train4, X_test4, y_test4, its = 10)
print(f"MAE: {minimo3}")
print(f"Parámetros: {pp3}")

iteration number 0
{'learning_rate': 0.640131488460717, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.19931923775962967, 'num_leaves': 220, 'min_data': 25, 'max_depth': 226, 'verbose': -1} 10000
MAE: 5970567.638243632
--------------------
iteration number 1
{'learning_rate': 0.6390634854423681, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.9911455201420419, 'num_leaves': 175, 'min_data': 5, 'max_depth': 40, 'verbose': -1} 10000
MAE: 3624676.455161284
--------------------
iteration number 2
{'learning_rate': 0.6609067898552232, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.8972459531054943, 'num_leaves': 161, 'min_data': 40, 'max_depth': 210, 'verbose': -1} 10000
MAE: 3700628.488743663
--------------------
iteration number 3
{'learning_rate': 0.616934775765042, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.7872557460811838, 'num_